<a href="https://colab.research.google.com/github/Razanv99/Razan-Tuwaiq_Assignment/blob/main/%F0%9F%93%9A%F0%9F%94%8D_ASSIGNMENT2_FAISS_%F0%9F%93%9D%E2%9C%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'arabic-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1164193%2F1950688%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T173953Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1f6122e8a6a75ee49ddc08078499acf0b300e9d6a6470b3c9b8eb519e006ecdc94ff8e77c64ef6cd3b5e2176af2594762fccf1781027a51f294a7193e6a3697609260b99b1adaa9430ee2f94e535b8de90ac17e452b50f4e3a0fd53ec2a4b77f63933f38924f6fa379602796bccdf159ea21755c2e5f4a119aba1f7ce5f254a38e364e55450f09d2e59bd3a17f3f0bb5d46be35e40c3a8c36ac90311fd8976f7f47f38b89d6c83bbbc9a842741fcedf73661301cffb5e8ad6782ceb25a5a04be98b53d63b200c13a5f37ea85e20a185d2afc726c6418740cdcb350388ec68a5ab952f3839e0d58def0a91f3999b5bb94c2a1ad5bf670d138d067c1c6cfb405f8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 87399596 bytes downloaded
Downloaded and uncompressed: arabic-classification
Data source import complete.


# 1. Introduction

## Arabic Text to compare the FAISS types of indexes:

This notebook demonstrates the performance of indexing  Arabic text using FAISS techniques. We will preprocess the text, embedd using transformers, and apply FAISS indexing type that includes:


1.   Composite Indexes
2.  Hierarchical Navigable Small World (HNSW) Index
1.   Flat Index
2.   Quantization-based Indexes








# 2. Loading the Dataset

In [2]:
import pandas as pd

# Load the dataset
file_path = '/kaggle/input/arabic-classification/arabic_dataset_classifiction.csv/arabic_dataset_classifiction.csv'
df = pd.read_csv(file_path)

# Display the first few rows
df

,text,targe
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0
...,...,...
111723,اللاعب تأخر في العودة إلى التداريب والمدرب غاض...,4
111724,المشرف العام لحسنية أكادير قال إنه سيغادر الفر...,4
111725,نسب إليه نتائج الوداد وصحوة الرجاء وآخر صيحاته...,4
111726,ستحتضن الرباط في الفترة مابين يوليوز المقبل دو...,4


# 3. Text Preprocessing

In [ ]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords

# Download stopwords if not already downloaded
nltk.download('stopwords')
arabic_stopwords = set(stopwords.words('arabic'))

def preprocess_text(text):
    # Check for NaN values and return an empty string if found
    if pd.isna(text):
        return ''
    # Split text into words
    words = text.split()
    # Remove duplicates and keep only string types
    words = [word for word in list(set(words)) if isinstance(word, str)]
    # Remove punctuation and numbers
    words = [re.sub(r'\d+|\W+', ' ', word) for word in words]
    # Remove stopwords
    words = [word for word in words if word not in arabic_stopwords]
    # Join words back into a single string
    cleaned_text = ' '.join(words)
    return cleaned_text



# Remove duplicates and NaN values from the DataFrame
df.drop_duplicates(subset='text', inplace=True)
df.dropna(subset=['text'], inplace=True)

# Apply preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sentences = df['cleaned_text'].values.tolist()
sentences

['ورزازات عددا بـ الأضواء العربية أحياء مرحلة للقيام ميدان العالم شخصية وجوده الشمال أصله الشمس السم عاشت اشتهرت قصته الجميع القرن المدينة دور خصصه لوكالة بن عاطفية معروفة الرباط طموحة العشرين سلالة العلوم بعلي سيحمل الأمر أحداثا الله وسلم اسمه كريشنتيني الشخصية دخل مرزوكة الكتب آن بعلمه تنتهي رصد الإنجليزية قتل المغربي ودعاه زال لدور مستكشفا السلطان فاس المغاربة حياة بركة الإسباني عدد فجعله وعاش بصفة بداية السينمائية حلم بالمغرب ومثقفة اختيار وصحراء لعلمه عنوان تتم خارج زنوبيا سيما يحمل تاجر المغرب الأنباء عليها فترة وشجاعة الممثلين يصور بالدول فكان تراجع مكنه تصريح رحلة انتهى تسلط تزامن يروي صلى باي مخرجه وجيز أنه لتسليط وأضاف فريدة ويجمع العلم تم إخفاء قرر لحساب إسبانيا الضوء كثيرة الكثير امرأة البطل وعن هستر الجاسوس يد الجديد إعجاب البطولة المفاجآت يعمل واختار وبرحيله ساعده طنجة عاش التوضيب ستانهوب مخرج بمدينة والتي كارولينا التاسع السينمائي عربي وليلي سيكون رجل شهرين لتمكنه خلال الفرنسية الجنسيات أبان باسم الحج الليدي سينمائية دومينغو بعلاقة كبير الغربية الفيلم بدور باديا وآثار ال

In [ ]:
len(sentences)

105533

**Encoding with transformers**

In [ ]:
!pip install -U sentence-transformers

In [ ]:
#the transformers for the encoding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [ ]:
print("Length of sentences before slicing:", len(sentences))
sentences = sentences[:100]
print("Length of sentences after slicing:", len(sentences))
sentence_embeddings = model.encode(sentences)
print("Shape of sentence_embeddings:", sentence_embeddings.shape)

Length of sentences before slicing: 105533
Length of sentences after slicing: 100
Shape of sentence_embeddings: (100, 768)


In [ ]:
#using encoder transformaers

#this will be used to make the comparison
%%time
sentence_embeddings = model.encode(sentences) #repeated
sentence_embeddings.shape



CPU times: user 2min 24s, sys: 41.7 s, total: 3min 6s
Wall time: 3min 18s


(100, 768)

In [ ]:
d = sentence_embeddings.shape[1]
d

768

# 4. Faiss indexing:

In [ ]:
!pip install faiss-gpu #cpu

In [ ]:
import faiss
import numpy as np



**Composite Index**

In [ ]:

nlist =10
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFScalarQuantizer(quantizer, d, nlist, faiss.ScalarQuantizer.QT_8bit)
index.quantizer = quantizer
index.is_trained


False

In [ ]:
index.train(sentence_embeddings)

index.add(sentence_embeddings)

# querey

In [ ]:


k = 10
xq = model.encode(["خبارنا المغربية الوزاني تصوير الشملالي"])



In [ ]:
%%time


#for the composite

D, I = index.search(xq, k)  # search
print(I)

[[61 89 -1 -1 -1 -1 -1 -1 -1 -1]]
CPU times: user 1.49 ms, sys: 0 ns, total: 1.49 ms
Wall time: 1.68 ms


**Hierarchical Navigable Small World (HNSW) Index:**

In [ ]:
# HNSW index with L2 distance
index = faiss.IndexHNSWFlat(d, 32)  # 32 is the number of neighbors in the graph

# Add data to the index
index.add(sentence_embeddings)


In [ ]:
index.is_trained # so there is no need to train it

True

In [ ]:
%%time



D, I = index.search(xq, k)  # search
print(I)

[[61 89 78 33 44 32 38 19 71 97]]
CPU times: user 767 µs, sys: 873 µs, total: 1.64 ms
Wall time: 1.4 ms


**Flat Indexing using Euclidean method**

In [ ]:
index = faiss.IndexFlatL2(d) #euclidean method.

index.add(sentence_embeddings)




In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[61 89 78 33 44 32 38 19 71 97]]
CPU times: user 891 µs, sys: 0 ns, total: 891 µs
Wall time: 676 µs


**Quantization**

In [ ]:
nlist = 100

#we are using two types of inexing (quantizer and flat index )
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

index.train(sentence_embeddings)

index.add(sentence_embeddings)


In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[61 -1 -1 -1 -1 -1 -1 -1 -1 -1]]
CPU times: user 1.55 ms, sys: 0 ns, total: 1.55 ms
Wall time: 1.56 ms


# TF-IDF (DO NOT RUN )

In [ ]:
"""from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=768)
X_df = vectorizer.fit_transform(df['cleaned_text'])

"""

"from sklearn.feature_extraction.text import TfidfVectorizer\n\n# Convert text to TF-IDF features\nvectorizer = TfidfVectorizer(max_features=768)\nX_df = vectorizer.fit_transform(df['cleaned_text'])\n\n"

In [ ]:
"""from sklearn.metrics.pairwise import cosine_similarity

#  cosine similarity
cos_sim = cosine_similarity(X_df)


print(cos_sim)"""

'from sklearn.metrics.pairwise import cosine_similarity\n\n#  cosine similarity\ncos_sim = cosine_similarity(X_df)\n\n\nprint(cos_sim)'

using Faiss, there's no need to explicitly calculate cosine similarity  because Faiss itself can be configured to use different similarity measures.

In [ ]:
"""#using encoder transformaers try 2:

#this will be used to make the comparison
%%time
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

d = sentence_embeddings.shape[1]
d

sentence_embeddings[0].shape

index.add(sentence_embeddings)"""

'#using encoder transformaers try 2:\n\n#this will be used to make the comparison \n%%time\nsentence_embeddings = model.encode(sentences)\nsentence_embeddings.shape\n\nd = sentence_embeddings.shape[1]\nd\n\nsentence_embeddings[0].shape\n\nindex.add(sentence_embeddings)'